In [ ]:
!pip install sentencepiece transformers

     |████████████████████████████████| 1.2 MB 5.2 MB/s 
     |████████████████████████████████| 2.9 MB 32.3 MB/s 
     |████████████████████████████████| 596 kB 48.4 MB/s 
     |████████████████████████████████| 56 kB 4.8 MB/s 
     |████████████████████████████████| 3.3 MB 35.3 MB/s 
     |████████████████████████████████| 895 kB 37.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

In [ ]:
def get_response(input_text,num_return_sequences,num_beams, max_len, min_len):
  batch = tokenizer(
      [input_text],
      truncation=True,
      padding='longest',
      max_length=60, 
      return_tensors="pt"
    ).to(torch_device)

  translated = model.generate(
      **batch,
      min_length=min_len,
      max_length=max_len,
      num_beams=num_beams, 
      num_return_sequences=num_return_sequences, 
      temperature=1.5
    )
  
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)

  return tgt_text

In [ ]:
num_beams = 10
num_return_sequences = 10
context = "I like trees , in particular oaks"

get_response(
    context,
    num_return_sequences,
    num_beams, 
    len(context),
    len(context.split())
)

['I enjoy trees in particular oaks.',
 'I enjoy trees, in particular oaks.',
 'In particular oaks, I like trees.',
 'In particular oaks, I like them.',
 'oaks are the trees I like the most.',
 'oaks are the trees that I like the most.',
 'I enjoy trees, particularly oaks.',
 'I enjoy oaks, in particular.',
 'I enjoy trees, especially oaks.',
 'In particular oaks, I enjoy them.']

In [ ]:
len(context)

33

In [ ]:
examples = [
  "I like westerns more than boardgames .",
  "I do not like biologists , I prefer waiters .",
  "I like birds , and hamsters too .",
  "I like parrots , and more specifically hamsters .",
  "I like lard , and apples too .",
  "I like earrings , except socks .",
  "I like shoes more than necklaces .",
  "He trusts his taste , except reports .",
  "He likes wisdom , except fear .",
  "I use nylon , except leather .",
  "I like clothes , and shirts too .",
  "I like chicken , and more specifically salmon .",
  "He trusts reports , except his taste .",
  "I like scientists more than caretakers .",
  "I like paintings , and chairs too .",
  "I use glass , but not PVC .",
  "I like dogs , an interesting type of giraffe .",
  "I like jewelry , but not blues .",
  "I use leather , and more specifically vinyl .",
  "I use glass more than nylon .",
  "I met caretakers , and more specifically factories .",
  "I like dogs , except parrots .",
  "I like caretakers more than workers .",
  "I like firs , and shrubs too .",
  "I like prosciutto , and more specifically broccoli .",
  "I use PVC , an interesting type of plastic .",
  "I like bushes more than pines .",
  "I like shoes , except earrings .",
  "I like Merlot , and more specifically beer .",
  "I use PVC , and leather too .",
  "I met professors , and more specifically factories .",
]

In [ ]:
for ex in examples:
  sents = get_response( ex, 5, 10,  len(ex), len(ex.split()))
  result = '\n\t'.join(s for s in sents if s != ex)
  print(f'------\n{ex}\n\t{result}\n--------')

------
I like westerns more than boardgames .
	I like westerns more than board games.
	I enjoy westerns more than board games.
	I like westerns more than games.
	I prefer westerns over board games.
	I like board games more than westerns.
--------
------
I do not like biologists , I prefer waiters .
	I don't like biologists and I prefer waiters.
	I don't like biologists and I like waiters.
	I don't like biologists and prefer waiters.
	I don't like biologists and I prefer the waiter.
	I don't like biologists, and I prefer waiters.
--------
------
I like birds , and hamsters too .
	I like birds and hamsters.
	I enjoy birds and hamsters.
	I like hamsters and birds.
	I love birds and hamsters.
	I like animals like birds and hamsters.
--------
------
I like parrots , and more specifically hamsters .
	I am a big fan of parrots and hamsters.
	I am fond of parrots and hamsters.
	I am a fan of parrots and hamsters.
	I like hamsters, and parrots.
	hamsters and parrots are my two favorite animals.

In [121]:
df[df.sentence.str.len() > 85]['sentence'].tolist()

['They like apprehensiveness more than they like premonition, but they trust premonition.',
 'Their appreciation of malevolence is greater than their appreciation of vindictiveness.',
 'Their appreciation of malevolence is higher than their appreciation of vindictiveness.',
 "He doesn't feel like he's in trouble, but on the other hand, he feels like he's in trouble.",
 "He doesn't feel like he's in trouble, but on the other hand he feels like he's in trouble.",
 'She likes to eat fruit that is not raisins, but her favorite is fruit that is inedible.',
 'Her appreciation of cruciferous vegetable is greater than her appreciation of cabbage.',
 'Her appreciation of cruciferous vegetable is greater than her head cabbage appreciation.',
 'Her appreciation of cruciferous vegetable is greater than her appreciation of cabbage head.',
 'They like savoy cabbage, also known as tempura, which is an interesting type of cabbage.',
 'My appreciation of cherry tomato is greater than my appreciation of

In [143]:
import pandas as pd
from pathlib import Path

files = Path('../data/data_experiments/combination_experiments/pegasus_added/').glob('*.csv')

# for f in files:
#     pd.read_csv(f).sample(frac=1).to_csv(f'{f.stem}-shuffled.csv', index=False)

In [144]:
faulty_sents = [
    '888-349-8884',
    'and chairs and chairs and',
    'and oaks and oaks and oaks'
    'and salmon and salmon and',
    '888-270-6611',
    'chair and chair and chair',
    'oak and oak and oak',
    'salmon and salmon and salmon'
]

In [145]:
for f in files:
    df = pd.read_csv(f)
    df.rename({'pegaus': 'pegasus'}, inplace=True)
    for test_item in faulty_sents:
        df = df[~df.sentence.str.contains(test_item)]
    df.to_csv(f, index=False)